## Default of Credit Card Clients

In [3]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.preprocessing import MinMaxScaler
df_original = pd.read_excel("input/default of credit card clients.xls",header=1)
df_original =df_original.rename({'PAY_0' : 'PAY_1','default payment next month':'default'},axis=1)
df = df_original.copy()
N_labels = [column for column in df_original.columns if column !='default']
split = StratifiedShuffleSplit(n_splits=1, test_size=0.25, random_state=42)
for train_index , test_index in split.split(df,df.iloc[:,[2,-1]]):
    df_train, label_train= df[N_labels].loc[train_index],df.default[train_index]
    df_test, label_test= df[N_labels].loc[test_index], df.default[test_index]
    

In [4]:
from sklearn.base import TransformerMixin, BaseEstimator
class default_transfomer(BaseEstimator,TransformerMixin):
    def __init__(self):
        self.columns=['ID', 'LIMIT_BAL', 'SEX', 'EDUCATION', 'MARRIAGE', 'AGE', 'PAY_1',
       'PAY_2', 'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6', 'BILL_AMT1', 'BILL_AMT2',
       'BILL_AMT3', 'BILL_AMT4', 'BILL_AMT5', 'BILL_AMT6', 'PAY_AMT1',
       'PAY_AMT2', 'PAY_AMT3', 'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6', 
       'USAGE_1', 'USAGE_2', 'USAGE_3', 'USAGE_4', 'USAGE_5',
       'DIFF_0', 'DIFF_1', 'DIFF_2', 'DIFF_3',
       'DIFF_4', 'log_LIMIT_BAL', 'log_BILL_AMT1', 'log_BILL_AMT2',
       'log_BILL_AMT3', 'log_BILL_AMT4', 'log_BILL_AMT5', 'log_BILL_AMT6',
       'log_PAY_AMT1', 'log_PAY_AMT2', 'log_PAY_AMT3', 'log_PAY_AMT4',
       'log_PAY_AMT5', 'log_PAY_AMT6', 'log_USAGE_1', 'log_USAGE_2',
       'log_USAGE_3', 'log_USAGE_4', 'log_USAGE_5', 'log_DIFF_0',
       'log_DIFF_1', 'log_DIFF_2', 'log_DIFF_3',
       'log_DIFF_4']
        self.log_columns=['ID', 'LIMIT_BAL', 'SEX', 'EDUCATION', 'MARRIAGE', 'AGE', 'PAY_1',
       'PAY_2', 'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6','log_LIMIT_BAL',
       'log_BILL_AMT1','log_BILL_AMT2', 'log_BILL_AMT3', 'log_BILL_AMT4', 'log_BILL_AMT5',
       'log_BILL_AMT6', 'log_PAY_AMT1', 'log_PAY_AMT2', 'log_PAY_AMT3',
       'log_PAY_AMT4', 'log_PAY_AMT5', 'log_PAY_AMT6', 'log_USAGE_1',
       'log_USAGE_2', 'log_USAGE_3', 'log_USAGE_4', 'log_USAGE_5',
       'log_DIFF_0', 'log_DIFF_1', 'log_DIFF_2', 'log_DIFF_3', 'log_DIFF_4']
    
    def fit(self):
        pass
    def log_pre (self,x):
        return np.log(1+abs(x))*np.sign(x)
    def log_pre_col (self,column):
        return column.map(lambda x:self.log_pre(x))
    
    def transform(self,df,y=None):
        #Usage
        df_usage = pd.DataFrame()
        df_usage['ID']=df.ID
        BILL = [column for column in df.columns if 'BILL' in column]
        PAY = [column for column in df.columns if 'PAY_AMT' in column]
        for i in range(5):
            df_usage[f"USAGE_{i+1}"] = df[BILL[i]]-(df[BILL[i+1]]-df[PAY[i]])
        #difference
        df_difference =pd.DataFrame()
        df_difference['ID'] = df.ID
        for i in range(5):
            df_difference[f"DIFF_{i}"] = df[BILL[i+1]]-df[PAY[i]]
        USAGE = [column for column in df_usage.columns if column !='ID']
        DIFF = [column for column in df_difference.columns if column !='ID']
        LIM =['LIMIT_BAL']
        ## Logarithmic Scaling attribute 
        df = pd.concat([df,df_usage.iloc[:,1:],df_difference.iloc[:,1:]],axis=1)
        df_log =pd.concat([df.ID,df[LIM+BILL+PAY+USAGE+DIFF].apply(self.log_pre_col,axis=0)],axis=1) #apply(function,axis=) map of those who use index/column as a index
        rename_dict = {}
        for i in df_log.iloc[:,1:].columns:
            rename_dict[i] = 'log_'+i
        df_log =df_log.rename(rename_dict,axis=1)
        self.BILL = BILL
        self.PAY =PAY
        self.DIFF = DIFF
        self.USAGE =USAGE
        return pd.concat([df,df_log[[column for column in df_log.columns if column !='ID']]],axis=1)

In [5]:
NAT = default_transfomer()

In [6]:
train_data = NAT.transform(df_train)

In [7]:
train_data

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_1,PAY_2,PAY_3,PAY_4,...,log_USAGE_1,log_USAGE_2,log_USAGE_3,log_USAGE_4,log_USAGE_5,log_DIFF_0,log_DIFF_1,log_DIFF_2,log_DIFF_3,log_DIFF_4
20349,20350,80000,1,1,2,25,2,2,-2,-2,...,11.277216,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6983,6984,300000,2,1,1,51,-1,-1,-1,-1,...,5.749393,6.722630,5.420535,0.000000,0.000000,-1.098612,0.000000,0.000000,0.000000,0.000000
17548,17549,60000,2,2,1,48,0,0,0,0,...,7.510431,7.095893,7.290975,10.476386,6.472346,10.799188,10.815268,10.827925,9.701494,9.698675
954,955,110000,2,2,1,46,-1,-1,-1,-1,...,7.787797,7.674153,7.678789,7.450080,8.445053,0.000000,0.000000,0.000000,0.000000,11.541911
12866,12867,20000,2,2,2,25,0,0,0,0,...,7.567346,8.178077,8.133294,7.483807,8.560636,9.812632,9.685207,9.531989,9.431562,9.492130
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22066,22067,160000,2,1,2,26,-1,-1,-1,0,...,7.886081,8.258422,8.017308,8.402904,7.886081,0.000000,0.000000,8.946114,8.347353,7.414573
19277,19278,80000,2,2,1,41,-1,-1,-1,-1,...,7.911324,7.426549,6.340359,8.126814,0.000000,0.000000,0.000000,0.000000,10.973049,10.973049
23975,23976,20000,1,1,2,25,0,0,0,0,...,9.775484,7.836370,7.996654,5.978886,8.172164,9.115920,8.957897,8.824531,9.036582,8.676758
12381,12382,500000,2,1,2,35,1,-1,-1,-1,...,0.000000,6.466145,10.616780,11.466588,7.809947,0.000000,-1.098612,-9.110741,-10.869273,0.000000


In [8]:
train_data.columns

Index(['ID', 'LIMIT_BAL', 'SEX', 'EDUCATION', 'MARRIAGE', 'AGE', 'PAY_1',
       'PAY_2', 'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6', 'BILL_AMT1', 'BILL_AMT2',
       'BILL_AMT3', 'BILL_AMT4', 'BILL_AMT5', 'BILL_AMT6', 'PAY_AMT1',
       'PAY_AMT2', 'PAY_AMT3', 'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6', 'USAGE_1',
       'USAGE_2', 'USAGE_3', 'USAGE_4', 'USAGE_5', 'DIFF_0', 'DIFF_1',
       'DIFF_2', 'DIFF_3', 'DIFF_4', 'log_LIMIT_BAL', 'log_BILL_AMT1',
       'log_BILL_AMT2', 'log_BILL_AMT3', 'log_BILL_AMT4', 'log_BILL_AMT5',
       'log_BILL_AMT6', 'log_PAY_AMT1', 'log_PAY_AMT2', 'log_PAY_AMT3',
       'log_PAY_AMT4', 'log_PAY_AMT5', 'log_PAY_AMT6', 'log_USAGE_1',
       'log_USAGE_2', 'log_USAGE_3', 'log_USAGE_4', 'log_USAGE_5',
       'log_DIFF_0', 'log_DIFF_1', 'log_DIFF_2', 'log_DIFF_3', 'log_DIFF_4'],
      dtype='object')

In [9]:
#print(pd.concat([train_data.SEX,label_train],axis=1))

In [10]:
scaler = MinMaxScaler()
ATs = [column for column in NAT.log_columns if column !='ID' if column !='LIMIT_BAL']
scaler.fit(train_data[ATs])
train_data_scaled = scaler.transform(train_data[ATs])
for train_index , test_index in split.split(train_data_scaled,pd.concat([train_data.SEX,label_train],axis=1)):
    X_train, y_train= train_data_scaled[train_index],label_train.iloc[train_index]
    X_test, y_test = train_data_scaled[test_index], label_train.iloc[test_index]

In [11]:
from sklearn.linear_model import LogisticRegressionCV
from sklearn.linear_model import LogisticRegression

In [12]:
clf = LogisticRegression(
                        max_iter=100000,random_state=42)
clf.fit(X_train,y_train)
print((clf.predict(X_train)==y_train).mean())
print((clf.predict(X_test)==y_test).mean())

0.8064592592592592
0.8069333333333333


In [13]:
for penalty in ['l1', 'l2', 'elasticnet', 'none']:
    try:
        print(penalty)
        clf = LogisticRegressionCV(
                                Cs=20,
                                cv=5,
                                penalty=penalty,
                                max_iter=800,
                                random_state=42)
        clf.fit(X_train,y_train)
        print((clf.predict(X_train)==y_train).mean())
        print((clf.predict(X_test)==y_test).mean())
    except:
        print(f'error for {penalty}')

l1
error for l1
l2
0.807762962962963
0.8065777777777777
elasticnet
error for elasticnet
none
error for none


In [14]:
for solver in {'lbfgs', 'liblinear', 'newton-cg', 'newton-cholesky', 'sag', 'saga'}:
    try:
        print(solver)
        clf = LogisticRegressionCV(
                                Cs=20,
                                cv=5,
                                solver=solver,
                                max_iter=800,
                                random_state=42)
        clf.fit(X_train,y_train)
        print((clf.predict(X_train)==y_train).mean())
        print((clf.predict(X_test)==y_test).mean())
    except:

        print(f"error for {solver}")

newton-cholesky
error for newton-cholesky
liblinear
0.808
0.8064
saga
0.8078222222222222
0.8065777777777777
sag
0.8079407407407407
0.8065777777777777
lbfgs
0.807762962962963
0.8065777777777777
newton-cg
0.8078222222222222
0.8065777777777777


In [15]:
scaler = MinMaxScaler()
ATs = [column for column in NAT.columns if column !='ID' ]
scaler.fit(train_data[ATs])
train_data_scaled = scaler.transform(train_data[ATs])
for train_index , test_index in split.split(train_data_scaled,pd.concat([train_data.SEX,label_train],axis=1)):
    X_train, y_train= train_data_scaled[train_index],label_train.iloc[train_index]
    X_test, y_test = train_data_scaled[test_index], label_train.iloc[test_index]

In [16]:

clf = LogisticRegression(
                        max_iter=100000,random_state=42)
clf.fit(X_train,y_train)
print((clf.predict(X_train)==y_train).mean())
print((clf.predict(X_test)==y_test).mean())

0.8087703703703704
0.8094222222222223


In [17]:
for penalty in ['l1', 'l2', 'elasticnet', 'none']:
    try:
        print(penalty)
        clf = LogisticRegressionCV(
                                Cs=20,
                                cv=5,
                                penalty=penalty,
                                max_iter=800,
                                random_state=42)
        clf.fit(X_train,y_train)
        print((clf.predict(X_train)==y_train).mean())
        print((clf.predict(X_test)==y_test).mean())
    except:
        print(f'error for {penalty}')

l1
error for l1
l2
0.8092444444444444
0.8096
elasticnet
error for elasticnet
none
error for none


In [18]:
for solver in {'lbfgs', 'liblinear', 'newton-cg', 'newton-cholesky', 'sag', 'saga'}:
    try:
        print(solver)
        clf = LogisticRegressionCV(
                                Cs=20,
                                cv=5,
                                solver=solver,
                                max_iter=800,
                                random_state=42)
        clf.fit(X_train,y_train)
        print((clf.predict(X_train)==y_train).mean())
        print((clf.predict(X_test)==y_test).mean())
    except:

        print(f"error for {solver}")

newton-cholesky
error for newton-cholesky
liblinear
0.8090666666666667
0.8097777777777778
saga
0.8091851851851852
0.8094222222222223
sag
0.8091851851851852
0.8094222222222223
lbfgs
0.8092444444444444
0.8096
newton-cg
0.8091851851851852
0.8094222222222223
